### grp

# Spark: The Definitive Guide

## PART 6: Advanced Analytics and Machine Learning 

## dataPaths

In [1]:
simpleML = '/Users/grp/sparkTheDefinitiveGuide/data/simple-ml/'
retailDataByDay = '/Users/grp/sparkTheDefinitiveGuide/data/retail-data/by-day/*.csv'
simpleMLInt = '/Users/grp/sparkTheDefinitiveGuide/data/simple-ml-integers/'
simpleMLScale = '/Users/grp/sparkTheDefinitiveGuide/data/simple-ml-scaling/'

## _Chapter #25 - Preprocessing and Feature Engineering_

-  Data Feature Format Per Use Case:
    -  Classification / Regression [column for label of type _Double_; column of type _Vector_ (dense or sparse) for features]
    -  Recommendation [column of users; column of items; column of ratings]
    -  Unsupervised [column of type _Vector_ (dense or sparse) for features]
    -  Graph [DF of vertices; DF of edges]   
    <br>
-  Transformers:
    -  best way to convert raw data (preprocessing) to needed format data for ML
    -  are functions that accept a DF as an argument and return a new DF as a response
    -  available in Spark package: **pyspark.ml.feature** / **org.apache.spark.ml.feature**   
    <br>
-  Estimators:
    -  when a transformation needs information (data) about the input column hence has to pass over (fit) the entire input column
    -  require to "fit" the transformer to dataset then call "transform" on resulting object producted from "fit"
    -  function must see all inputs from dataset to select a mapping of inputs (ex: StringIndexer)   
    <br>
-  High Level Transfomers (ex: RFormula):
    -  handles categorical inputs via one-hot encoding (converts set of values into set of binary columns)
    -  handles numeric columns as casted as _Double_
    -  handles string label (target variable) columns as transformed to _Double_ with StringIndexer   
    <br>
-  Continuous Features:
    -  _Bucketizer_
    -  _QuantileDiscretizer_
    -  _StandardScaler_
    -  _MinMaxScaler_
    -  _MaxAbsScaler_
    -  _ElementwiseProduct_
    -  _Normalizer_     
    <br>
-  Categorical Features:
    -  _StringIndexer_
    -  _OneHotEncoder_ (replaced with OneHotEncoderEstimator in Spark 2.3)
    -  _VectorIndexer_
    -  _IndexToString_
    -  _Tokenizer_
    -  _RegexTokenizer_
    -  _StopWordsRemover_
    -  _NGram_     
    <br>
-  Converting Words into Numerical Representation:
    -  _CountVectorizer_
    -  _HashingTF_
    -  _IDF_
    -  _Word2Vec_      
    <br>
-  Feature Manipulation:
    -  _PCA_
    -  _Interaction_
    -  _Polynomial Expansion_   
    <br>
-  Feature Selection:
    -  _ChiSqSelector_

### _Chapter #25 Exercises (FE)_

In [2]:
sales = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(retailDataByDay)\
.coalesce(5)\
.where("Description IS NOT NULL")

fakeIntDF = spark.read.parquet(simpleMLInt)
simpleDF = spark.read.json(simpleML)
scaleDF = spark.read.parquet(simpleMLScale)

In [3]:
sales.cache() # cache into memory for efficient re-use
sales.show(3)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows



In [4]:
print(fakeIntDF.show(3))
print(simpleDF.show(3))
print(scaleDF.show(3))

+----+----+----+
|int1|int2|int3|
+----+----+----+
|   1|   2|   3|
|   7|   8|   9|
|   4|   5|   6|
+----+----+----+

None
+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 3 rows

None
+---+--------------+
| id|      features|
+---+--------------+
|  0|[1.0,0.1,-1.0]|
|  1| [2.0,1.1,1.0]|
|  0|[1.0,0.1,-1.0]|
+---+--------------+
only showing top 3 rows

None


### _RFormula [Estimator] Example_

In [5]:
from pyspark.ml.feature import RFormula

In [6]:
supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show(3, False)

# type Row format
for i in supervised.fit(simpleDF).transform(simpleDF).take(3): print(i)

+-----+----+------+------------------+--------------------------------------------------------------------+-----+
|color|lab |value1|value2            |features                                                            |label|
+-----+----+------+------------------+--------------------------------------------------------------------+-----+
|green|good|1     |14.386294994851129|(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])|1.0  |
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])      |0.0  |
|blue |bad |12    |14.386294994851129|(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])    |0.0  |
+-----+----+------+------------------+--------------------------------------------------------------------+-----+
only showing top 3 rows

Row(color='green', lab='good', value1=1, value2=14.386294994851129, features=SparseVector(10, {1: 1.0, 2: 1.0, 3: 14.3863, 5: 1.0, 8: 14.3863}), label=1.0)
Row(color='blue', lab

### _SQLTransformer [Transformer] Example_

In [7]:
from pyspark.ml.feature import SQLTransformer

In [8]:
# "__THIS__" represents the underlying table of the input dataset
basicTransformation = SQLTransformer()\
  .setStatement("""
    SELECT sum(Quantity), count(*), CustomerID
    FROM __THIS__
    GROUP BY CustomerID
  """)
basicTransformation.transform(sales).show(3)

+-------------+--------+----------+
|sum(Quantity)|count(1)|CustomerID|
+-------------+--------+----------+
|          119|      62|   14452.0|
|          440|     143|   16916.0|
|          630|      72|   17633.0|
+-------------+--------+----------+
only showing top 3 rows



### _VectorAssembler [Transformer] Example_

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
# concatenates all features into long vector to pass into an estimator
# takes as input a number of columns of type Boolean, Double, or Vector
va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

+----+----+----+--------------------------------------------+
|int1|int2|int3|VectorAssembler_4ab98a8d8cde298cb88d__output|
+----+----+----+--------------------------------------------+
|   1|   2|   3|                               [1.0,2.0,3.0]|
|   7|   8|   9|                               [7.0,8.0,9.0]|
|   4|   5|   6|                               [4.0,5.0,6.0]|
+----+----+----+--------------------------------------------+



### _Continuous Feature Examples_:
-  Bucketing:
    -  convert continuous data to categorical data via "buckets or bins"
        -  Transfomers:
            -  _Bucketizer_
            -  _QuantileDiscretizer_   
            <br>
-  Scaling and Normalization:
    -  normalization helps with transforming data so each point's value is a representation of its distance from the mean of that column
    -  scaling helps with keeping data on the same scale so that values can easily be compared to one another for sensitive variations
        -  Estimators:
            -  _StandardScaler_
            -  _MinMaxScaler_
            -  _MaxAbsScaler_
        -  Transformers:
            -  _ElementwiseProduct_
            -  _Normalizer_

In [11]:
contDF = spark.range(20).selectExpr("cast(id as double)")
contDF.show(3)

+---+
| id|
+---+
|0.0|
|1.0|
|2.0|
+---+
only showing top 3 rows



In [12]:
from pyspark.ml.feature import Bucketizer

In [13]:
# "bins" continuous features into category "buckets"
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show(10)

+---+---------------------------------------+
| id|Bucketizer_4cb6bcff679ecbf0bcfe__output|
+---+---------------------------------------+
|0.0|                                    0.0|
|1.0|                                    0.0|
|2.0|                                    0.0|
|3.0|                                    0.0|
|4.0|                                    0.0|
|5.0|                                    1.0|
|6.0|                                    1.0|
|7.0|                                    1.0|
|8.0|                                    1.0|
|9.0|                                    1.0|
+---+---------------------------------------+
only showing top 10 rows



In [14]:
from pyspark.ml.feature import QuantileDiscretizer

In [15]:
# splits data based on percentiles
bucketer = QuantileDiscretizer().setNumBuckets(5).setInputCol("id")
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show(10)

+---+------------------------------------------------+
| id|QuantileDiscretizer_4d65a9096c72d69e239e__output|
+---+------------------------------------------------+
|0.0|                                             0.0|
|1.0|                                             0.0|
|2.0|                                             0.0|
|3.0|                                             1.0|
|4.0|                                             1.0|
|5.0|                                             1.0|
|6.0|                                             1.0|
|7.0|                                             2.0|
|8.0|                                             2.0|
|9.0|                                             2.0|
+---+------------------------------------------------+
only showing top 10 rows



In [16]:
from pyspark.ml.feature import StandardScaler

In [17]:
# scales input column according to range of values in that column to have zero mean and a variance of 1 in each dimension
# transforms a dataset of Vector rows then normalizes each feature to have unit standard deviation and/or zero mean
ss = StandardScaler().setInputCol("features")
ss.fit(scaleDF).transform(scaleDF).show(3, truncate=False)

+---+--------------+------------------------------------------------------------+
|id |features      |StandardScaler_41d69ae08066183d13f3__output                 |
+---+--------------+------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
+---+--------------+------------------------------------------------------------+
only showing top 3 rows



In [18]:
from pyspark.ml.feature import MinMaxScaler

In [19]:
# scales the values into a vector based on min/max boundry
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show(3)

+---+--------------+-----------------------------------------+
| id|      features|MinMaxScaler_4cb7b624b977c83e558d__output|
+---+--------------+-----------------------------------------+
|  0|[1.0,0.1,-1.0]|                            [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                            [7.5,5.5,7.5]|
|  0|[1.0,0.1,-1.0]|                            [5.0,5.0,5.0]|
+---+--------------+-----------------------------------------+
only showing top 3 rows



In [20]:
from pyspark.ml.feature import MaxAbsScaler

In [21]:
# scales the values into a vector by dividing each value by the max absolute value in feature
maScaler = MaxAbsScaler().setInputCol("features")
fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show(3, truncate=False)

+---+--------------+-------------------------------------------------------------+
|id |features      |MaxAbsScaler_4c60b9b27cc21512637d__output                    |
+---+--------------+-------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.3333333333333333,0.009900990099009901,-0.3333333333333333]|
|1  |[2.0,1.1,1.0] |[0.6666666666666666,0.10891089108910892,0.3333333333333333]  |
|0  |[1.0,0.1,-1.0]|[0.3333333333333333,0.009900990099009901,-0.3333333333333333]|
+---+--------------+-------------------------------------------------------------+
only showing top 3 rows



In [22]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors

In [23]:
# scales the values into a vector by an arbitrary value
scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct().setScalingVec(scaleUpVec).setInputCol("features")
scalingUp.transform(scaleDF).show(3, truncate=False)

+---+--------------+-----------------------------------------------+
|id |features      |ElementwiseProduct_43fea42384f05fefbbe5__output|
+---+--------------+-----------------------------------------------+
|0  |[1.0,0.1,-1.0]|[10.0,1.5,-20.0]                               |
|1  |[2.0,1.1,1.0] |[20.0,16.5,20.0]                               |
|0  |[1.0,0.1,-1.0]|[10.0,1.5,-20.0]                               |
+---+--------------+-----------------------------------------------+
only showing top 3 rows



In [24]:
from pyspark.ml.feature import Normalizer

In [25]:
# scales the values into a vector via a unit norm (parameter 'p')
# helps standardize input data and improve the behavior of learning algorithms
manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show(3, truncate=False)

+---+--------------+---------------------------------------------------------------+
|id |features      |Normalizer_4f40bbb6be281d439ed9__output                        |
+---+--------------+---------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.47619047619047616,0.047619047619047616,-0.47619047619047616]|
|1  |[2.0,1.1,1.0] |[0.48780487804878053,0.26829268292682934,0.24390243902439027]  |
|0  |[1.0,0.1,-1.0]|[0.47619047619047616,0.047619047619047616,-0.47619047619047616]|
+---+--------------+---------------------------------------------------------------+
only showing top 3 rows



### _Categorical Feature Examples_:
-  Indexing:
    -  converts categorical varaible to numerical variables
        -  Estimators:
            -  _StringIndexer_
            -  _OneHotEncoder_ (replaced with OneHotEncoderEstimator in Spark 2.3)
            -  _VectorIndexer_
        -  Transformers:
            -  _IndexToString_   
            <br>
-  Text Data Transformers:
    -  parses text for analysis
        -  Transformers:
            -  _Tokenizer_
            -  _RegexTokenizer_
            -  _StopWordsRemover_
            -  _NGram_

In [26]:
from pyspark.ml.feature import StringIndexer

In [27]:
# maps strings to different numerical ID
# creates metadata attached to DF specifying what inputs correspond to what outputs 
lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show(3)

# numeric input column string indexer 
valIndexer = StringIndexer().setInputCol("value1").setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show(5)

# example of skipping row if the input value was not a value seen during training
'''
valIndexer.setHandleInvalid("skip")
valIndexer.fit(simpleDF).setHandleInvalid("skip")
'''

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|labelInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
+-----+----+------+------------------+--------+
only showing top 3 rows

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|valueInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     2.0|
| blue| bad|     8|14.386294994851129|     4.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     5.0|
|green|good|    12|14.386294994851129|     0.0|
+-----+----+------+------------------+--------+
only showing top 5 rows



'\nvalIndexer.setHandleInvalid("skip")\nvalIndexer.fit(simpleDF).setHandleInvalid("skip")\n'

In [28]:
from pyspark.ml.feature import IndexToString

In [29]:
# map numeric IDs back to original values via indexed metadata
labelReverse = IndexToString().setInputCol("labelInd")
labelReverse.transform(idxRes).show(3)

+-----+----+------+------------------+--------+------------------------------------------+
|color| lab|value1|            value2|labelInd|IndexToString_4ae2b6d27efc340a63f2__output|
+-----+----+------+------------------+--------+------------------------------------------+
|green|good|     1|14.386294994851129|     1.0|                                      good|
| blue| bad|     8|14.386294994851129|     0.0|                                       bad|
| blue| bad|    12|14.386294994851129|     0.0|                                       bad|
+-----+----+------+------------------+--------+------------------------------------------+
only showing top 3 rows



In [30]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors

In [31]:
# helps index categorical features in datasets of vectors
# automatically finds categorical features inside of input vectors and converts them to categorical features
idxIn = spark.createDataFrame([\
(Vectors.dense(1, 2, 3),1),
(Vectors.dense(2, 5, 6),2),
(Vectors.dense(1, 8, 9),3)])\
.toDF("features", "label")

indxr = VectorIndexer()\
.setInputCol("features")\
.setOutputCol("idxed")\
.setMaxCategories(2)

indxr.fit(idxIn).transform(idxIn).show(3)

+-------------+-----+-------------+
|     features|label|        idxed|
+-------------+-----+-------------+
|[1.0,2.0,3.0]|    1|[0.0,2.0,3.0]|
|[2.0,5.0,6.0]|    2|[1.0,5.0,6.0]|
|[1.0,8.0,9.0]|    3|[0.0,8.0,9.0]|
+-------------+-----+-------------+



In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [33]:
# converts each distinct category value to a boolean flag (1 or 0)
# vital method because just indexing does not always represent categorical variables correctly when being processed by algorithms
# ex: algorithm will mathematically treat blue (2.0) > green (1.0) which is incorrect based on feature
lblIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder().setInputCol("colorInd")
ohe.transform(colorLab).show(7)

# type Row format
for i in ohe.transform(colorLab).take(7): print(i)

+-----+--------+------------------------------------------+
|color|colorInd|OneHotEncoder_47d185be83f3013989bc__output|
+-----+--------+------------------------------------------+
|green|     1.0|                             (2,[1],[1.0])|
| blue|     2.0|                                 (2,[],[])|
| blue|     2.0|                                 (2,[],[])|
|green|     1.0|                             (2,[1],[1.0])|
|green|     1.0|                             (2,[1],[1.0])|
|green|     1.0|                             (2,[1],[1.0])|
|  red|     0.0|                             (2,[0],[1.0])|
+-----+--------+------------------------------------------+
only showing top 7 rows

Row(color='green', colorInd=1.0, OneHotEncoder_47d185be83f3013989bc__output=SparseVector(2, {1: 1.0}))
Row(color='blue', colorInd=2.0, OneHotEncoder_47d185be83f3013989bc__output=SparseVector(2, {}))
Row(color='blue', colorInd=2.0, OneHotEncoder_47d185be83f3013989bc__output=SparseVector(2, {}))
Row(color='green', c

In [34]:
from pyspark.ml.feature import Tokenizer

In [35]:
# splits text into array of words based on separated (default is a whitespace) parser
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(3, truncate=False)

+-------------------------------+-------------------------------------+
|Description                    |DescOut                              |
+-------------------------------+-------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|
+-------------------------------+-------------------------------------+
only showing top 3 rows



In [36]:
from pyspark.ml.feature import RegexTokenizer

In [37]:
# splits text into array of words based on custom separated parser
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(3, False)

# set "gaps" to False to return output values matching the pattern
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setGaps(False)\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(3, False)

+-------------------------------+-------------------------------------+
|Description                    |DescOut                              |
+-------------------------------+-------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|
+-------------------------------+-------------------------------------+
only showing top 3 rows

+-------------------------------+------------+
|Description                    |DescOut     |
+-------------------------------+------------+
|RABBIT NIGHT LIGHT             |[ ,  ]      |
|DOUGHNUT LIP GLOSS             |[ ,  ,  ]   |
|12 MESSAGE CARDS WITH ENVELOPES|[ ,  ,  ,  ]|
+-------------------------------+------------+
only showing top 3 rows



In [38]:
from pyspark.ml.feature import StopWordsRemover

In [39]:
# filters out stop words that are not relevant to analysis to reduce noise within ML dataset
englishStopWords = StopWordsRemover.loadDefaultStopWords("english") # loads list of default stop words
stops = StopWordsRemover()\
.setStopWords(englishStopWords)\
.setInputCol("DescOut")
stops.transform(tokenized).show(3, truncate=False)

+-------------------------------+-------------------------------------+---------------------------------------------+
|Description                    |DescOut                              |StopWordsRemover_4beab8172e52ec7c19e8__output|
+-------------------------------+-------------------------------------+---------------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |[rabbit, night, light]                       |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |[doughnut, lip, gloss]                       |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|[12, message, cards, envelopes]              |
+-------------------------------+-------------------------------------+---------------------------------------------+
only showing top 3 rows



In [40]:
from pyspark.ml.feature import NGram

In [41]:
# word combinations to capture sentence structure and word patterns
unigram = NGram().setInputCol("DescOut").setN(1) # n-gram of length 1
bigram = NGram().setInputCol("DescOut").setN(2) # n-gram of length 2
trigram = NGram().setInputCol("DescOut").setN(3) # n-gram of length 3
unigram.transform(tokenized.select("DescOut")).show(3, False)
bigram.transform(tokenized.select("DescOut")).show(3, False)
trigram.transform(tokenized.select("DescOut")).show(3, False)

+-------------------------------------+-------------------------------------+
|DescOut                              |NGram_4788880d83b933cb73a9__output   |
+-------------------------------------+-------------------------------------+
|[rabbit, night, light]               |[rabbit, night, light]               |
|[doughnut, lip, gloss]               |[doughnut, lip, gloss]               |
|[12, message, cards, with, envelopes]|[12, message, cards, with, envelopes]|
+-------------------------------------+-------------------------------------+
only showing top 3 rows

+-------------------------------------+-------------------------------------------------------+
|DescOut                              |NGram_4529a99e840e07f126f7__output                     |
+-------------------------------------+-------------------------------------------------------+
|[rabbit, night, light]               |[rabbit night, night light]                            |
|[doughnut, lip, gloss]               |[dough

### _Converting Words into Numerical Representations Examples_:
-  takes transformed text and converts into numerical vectors
-  measures whether or not each row contains a given word
-  every row is a "document" and every word is a "term" and the total collection of all terms is the "vocabulary"
    -  Estimators:
        -  _CountVectorizer_
        -  _HashingTF_
        -  _IDF_
        -  _Word2Vec_

## CountVectorizer:
-  outputs the counts of a term in a document
    -  FIT Process:
        -  finds set of words in all documents (row) and then counts occurrences of words in the documents
    -  TRANSFORM Process:
        -  counts occurences of word in each row of DF column and outputs vector with terms that occur in that row
    -  Parameters:
        -  minTF:
            -  "minimum term frequency" for the term to be included in the vocabulary
            -  helps with removing rare words from the vocabulary
        -  minDF:
            -  "minimum - of documents" a term must appear before being included in the vocabulary"
            -  helps with removing rare words from the vocabulary
        - vocabSize:
            -  "total maximum vocabulary size"
-  How To Read The "countVec" Sparse Vector:
    -  ("vocabulary size", ["index of word in the vocabulary"], ["count of that particular word"])

In [42]:
from pyspark.ml.feature import CountVectorizer

In [43]:
cv = CountVectorizer()\
.setInputCol("DescOut")\
.setOutputCol("countVec")\
.setVocabSize(500)\
.setMinTF(1)\
.setMinDF(2)
fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show(3, False)

# type Row format
for i in fittedCV.transform(tokenized).take(3): print(i)

+-------------------------------+-------------------------------------+---------------------------------+
|Description                    |DescOut                              |countVec                         |
+-------------------------------+-------------------------------------+---------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |(500,[150,185,212],[1.0,1.0,1.0])|
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |(500,[462,463,491],[1.0,1.0,1.0])|
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|(500,[35,41,166],[1.0,1.0,1.0])  |
+-------------------------------+-------------------------------------+---------------------------------+
only showing top 3 rows

Row(Description='RABBIT NIGHT LIGHT', DescOut=['rabbit', 'night', 'light'], countVec=SparseVector(500, {150: 1.0, 185: 1.0, 212: 1.0}))
Row(Description='DOUGHNUT LIP GLOSS ', DescOut=['doughnut', 'lip', 'gloss'], countVec=SparseVecto

## TF-IDF [Term Frequency-Inverse Document Frequency]:
-  measures how often a word occurs in each document; weighted according to how many documents that word appears
-  words that occur in a few documents have higher weights than words that occur in many documents
-  ex: term like "the" appearing in every document within corpus will be weighted extremely low
-  helps find words that share similar topics
    -  How To Read The "IDFOut" Sparse Vector:
        - ("total vocabulary size", ["hash of every word appearing in document"], ["weight of each term"])

In [44]:
from pyspark.ml.feature import HashingTF, IDF

In [45]:
tfIdfIn = tokenized\
.where("array_contains(DescOut, 'red')")\
.select("DescOut")\
.limit(7)
tfIdfIn.show(7, False)

# hash each word and convert to numerical representation
# (similar to CountVectorizer; maps input words to index [words can be mapped to same index])
tf = HashingTF()\
.setInputCol("DescOut")\
.setOutputCol("TFOut")\
.setNumFeatures(10000)

# weight each word in vocabulary via IDF
idf = IDF()\
.setInputCol("TFOut")\
.setOutputCol("IDFOut")\
.setMinDocFreq(2)

idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(7, False)

# type Row format
for i in idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).take(7): print(i)

+---------------------------------------+
|DescOut                                |
+---------------------------------------+
|[gingham, heart, , doorstop, red]      |
|[red, floral, feltcraft, shoulder, bag]|
|[alarm, clock, bakelike, red]          |
|[pin, cushion, babushka, red]          |
|[red, retrospot, mini, cases]          |
|[red, kitchen, scales]                 |
|[gingham, heart, , doorstop, red]      |
+---------------------------------------+

+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|DescOut                                |TFOut                                                   |IDFOut                                                                                                              |
+---------------------------------------+--------------------------------------------------------+-------

## Word2Vec:
-  computes a vector representation of a set of words
-  aims to have similar words in the vector space to make generalizations
-  captures relationships between words based on their semantics
    -  Process:
        -  uses technique called "skip-grams" to convert sentence of words into vector representation (vectorSize) ...
        -  builds vocabulary
        -  maps each word to a unique fixed-size vector
        -  transforms each document into a vector using the average of all words in the document
        -  for each sentence (row) it removes a token / trains model to predict missing token in the "n-gram" representation

In [46]:
from pyspark.ml.feature import Word2Vec

In [47]:
# input data: each row is a bag of words from a sentence or document
documentDF = spark.createDataFrame([\
("Hi I heard about Spark".split(" "), ),
("I wish Java could use case classes".split(" "), ),
("Logistic regression models are neat".split(" "), )], ["text"])

# learn a mapping from words to vectors
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector))) 

Text: [Hi, I, heard, about, Spark] => 
Vector: [0.08242294350638986,-0.005830296874046326,-0.05622698366641998]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.03722663476530994,-0.0021124311855861117,-0.013124272493379456]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.08365558385848999,0.00822269544005394,-0.040985722094774246]



### _Feature Manipulation Examples_:
-  automated mathematical means used to:
    -  expand the input feature vectors
    -  reduce input feature vectors to a lower number of dimensions
        -  Estimators:
            -  _PCA_
        -  Transformers:
            -  _Interaction_
            -  _PolynomialExpansion_

## PCA [Principal Component Analysis]:
-  helps find the most import aspects of the data (principal components)
-  helps to create a smaller set of more relevant features as input to model
    -  Process:
        -  changes feature representation of data by creating a new set of features called "aspects":
        -  only include combination of original features as input to model
    -  Use Case:
        -  helpful for large input datasets with too many features
    -  Parameter:
        -  k:
            -  specifies "number of output features to create" (should be much smaller than input vectors' dimension)

In [48]:
from pyspark.ml.feature import PCA

In [49]:
pca = PCA().setInputCol("features").setK(2)
pca.fit(scaleDF).transform(scaleDF).show(5, False)

+---+--------------+------------------------------------------+
|id |features      |PCA_44dda20e4cb44486b5e6__output          |
+---+--------------+------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|1  |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060149758]|
+---+--------------+------------------------------------------+



## Interaction:
-  creates an interaction between 2 variables having an important correlation
-  only available via Scala API however can be developed via RFormula
    -  Process:
        -  multiplies the 2 features together

## Polynomial Expansion:
-  used to see interactions between particular features when unsure which interactions to consider
    -  Process:
        -  takes each value in feature vector and multiplies it by every other value in feature vector
        -  Ex:
            -  3 input features with degree-2 polynomial will product 9 output features (3X3)
            -  3 input features with degree-3 polynomial will product 27 output features (3X3X3)

In [50]:
from pyspark.ml.feature import PolynomialExpansion

In [51]:
pe = PolynomialExpansion().setInputCol("features").setDegree(2)
pe.transform(scaleDF).show(3, False)

+---+--------------+---------------------------------------------------------+
|id |features      |PolynomialExpansion_469b93cc7a5a801887ce__output         |
+---+--------------+---------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[1.0,1.0,0.1,0.1,0.010000000000000002,-1.0,-1.0,-0.1,1.0]|
|1  |[2.0,1.1,1.0] |[2.0,4.0,1.1,2.2,1.2100000000000002,1.0,2.0,1.1,1.0]     |
|0  |[1.0,0.1,-1.0]|[1.0,1.0,0.1,0.1,0.010000000000000002,-1.0,-1.0,-0.1,1.0]|
+---+--------------+---------------------------------------------------------+
only showing top 3 rows



### _Feature Selection Examples_:
-  method to select smaller subset of features for training from large range of available features
-  many features might be correlated, but using to many features might lead to overfitting
    -  Estimators:
        -  _ChiSqSelector_

## ChiSqSelector:
-  helps identify features that are relevant to the dependent variable
-  hence drops the uncorrelated features
-  best suited for categorical data to reduce number of features as input to model
    -  Parameters:
        -  numTopFeatures:
            -  selects fixed number of features ordered by p-value
        -  percentile:
            -  takes proportion of the input features
        -  fpr:
            -  sets p-value cut off

In [52]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer

In [53]:
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")

tokenized = tkn\
.transform(sales.select("Description", "CustomerId"))\
.where("CustomerId IS NOT NULL")

prechi = fittedCV.transform(tokenized)\
.where("CustomerId IS NOT NULL")

chisq = ChiSqSelector()\
.setFeaturesCol("countVec")\
.setLabelCol("CustomerId")\
.setNumTopFeatures(2)

chisq.fit(prechi).transform(prechi).drop("customerId", "Description", "DescOut").show(3, False)

# type Row format
for i in chisq.fit(prechi).transform(prechi).drop("customerId", "Description", "DescOut").take(3): print(i)

+---------------------------------+------------------------------------------+
|countVec                         |ChiSqSelector_41db9d7f0a1256d2158e__output|
+---------------------------------+------------------------------------------+
|(500,[150,185,212],[1.0,1.0,1.0])|(2,[],[])                                 |
|(500,[462,463,491],[1.0,1.0,1.0])|(2,[],[])                                 |
|(500,[35,41,166],[1.0,1.0,1.0])  |(2,[],[])                                 |
+---------------------------------+------------------------------------------+
only showing top 3 rows

Row(countVec=SparseVector(500, {150: 1.0, 185: 1.0, 212: 1.0}), ChiSqSelector_41db9d7f0a1256d2158e__output=SparseVector(2, {}))
Row(countVec=SparseVector(500, {462: 1.0, 463: 1.0, 491: 1.0}), ChiSqSelector_41db9d7f0a1256d2158e__output=SparseVector(2, {}))
Row(countVec=SparseVector(500, {35: 1.0, 41: 1.0, 166: 1.0}), ChiSqSelector_41db9d7f0a1256d2158e__output=SparseVector(2, {}))


### _Feature Selection Examples_:
-  persisting transformers/estimators
-  custom transformers/estimators

In [54]:
from pyspark.ml.feature import PCAModel

In [55]:
fittedPCA = pca.fit(scaleDF)
fittedPCA.write().overwrite().save("/Users/grp/sparkTheDefinitiveGuide/tmp/fittedPCA")

loadedPCA = PCAModel.load("/Users/grp/sparkTheDefinitiveGuide/tmp/fittedPCA")
loadedPCA.transform(scaleDF).show(3, False)

+---+--------------+------------------------------------------+
|id |features      |PCA_44dda20e4cb44486b5e6__output          |
+---+--------------+------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
+---+--------------+------------------------------------------+
only showing top 3 rows



In [56]:
'''
import org.apache.spark.ml.UnaryTransformer
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable,Identifiable}
import org.apache.spark.sql.types.{ArrayType, StringType, DataType}
import org.apache.spark.ml.param.{IntParam, ParamValidators}

class MyTokenizer(override val uid: String)
  extends UnaryTransformer[String, Seq[String],
    MyTokenizer] with DefaultParamsWritable {

  def this() = this(Identifiable.randomUID("myTokenizer"))

  val maxWords: IntParam = new IntParam(this, "maxWords",
    "The max number of words to return.",
  ParamValidators.gtEq(0))

  def setMaxWords(value: Int): this.type = set(maxWords, value)

  def getMaxWords: Integer = $(maxWords)

  override protected def createTransformFunc: String => Seq[String] = (
    inputString: String) => {
      inputString.split("\\s").take($(maxWords))
  }

  override protected def validateInputType(inputType: DataType): Unit = {
    require(
      inputType == StringType, s"Bad input type: $inputType. Requires String.")
  }

  override protected def outputDataType: DataType = new ArrayType(StringType,
    true)
}

// this will allow you to read it back in by using this object.
object MyTokenizer extends DefaultParamsReadable[MyTokenizer]

val myT = new MyTokenizer().setInputCol("someCol").setMaxWords(2)
myT.transform(Seq("hello world. This text won't show.").toDF("someCol")).show()
'''

'\nimport org.apache.spark.ml.UnaryTransformer\nimport org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable,Identifiable}\nimport org.apache.spark.sql.types.{ArrayType, StringType, DataType}\nimport org.apache.spark.ml.param.{IntParam, ParamValidators}\n\nclass MyTokenizer(override val uid: String)\n  extends UnaryTransformer[String, Seq[String],\n    MyTokenizer] with DefaultParamsWritable {\n\n  def this() = this(Identifiable.randomUID("myTokenizer"))\n\n  val maxWords: IntParam = new IntParam(this, "maxWords",\n    "The max number of words to return.",\n  ParamValidators.gtEq(0))\n\n  def setMaxWords(value: Int): this.type = set(maxWords, value)\n\n  def getMaxWords: Integer = $(maxWords)\n\n  override protected def createTransformFunc: String => Seq[String] = (\n    inputString: String) => {\n      inputString.split("\\s").take($(maxWords))\n  }\n\n  override protected def validateInputType(inputType: DataType): Unit = {\n    require(\n      inputType == StringTyp

### grp